In [7]:
from PyQt5.QtWidgets import *
from PyQt5.QtMultimedia import *
from PyQt5.QtMultimediaWidgets import *
from PyQt5.QtCore import QThread, Qt, pyqtSignal, pyqtSlot, QPoint
from PyQt5.QtGui import QImage, QPixmap, QPainter
import os
import sys
import time
import cv2
import math
import numpy as np
import mysql.connector
from scipy.signal import convolve2d
from easygui import *
import pywhatkit
import webbrowser
  

# Face Detection
# https://realpython.com/face-recognition-with-python/#cascades-in-practice

def detectFace(image_path):
    faceCascade = cv2.CascadeClassifier('Haarcascade_frontalface_default.xml')
    
    # Read the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(gray, 1.1, 9)
    
    if len(faces) > 0:
        valid = '1'
    else:
        valid = '0'
    
    return valid

# avarage color of image
# https://www.delftstack.com/howto/python/opencv-average-color-of-image/

def avgColor(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    average_color_row = np.average(image, axis=0)
    average_color = np.average(average_color_row, axis=0)

    return average_color

#NOISE
def estimate_noise(img):
  image = cv2.imread(img)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  H, W = image.shape

  M = [[1, -2, 1],
       [-2, 4, -2],
       [1, -2, 1]]

  sigma = np.sum(np.sum(np.absolute(convolve2d(image, M))))
  sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))

  return sigma

#DATABASE https://www.w3schools.com/python/python_mysql_getstarted.asp
#https://pynative.com/python-mysql-database-connection/

mydb = mysql.connector.connect(
  host="localhost",
  user="bob",
  password="123456",
  database="music"
)

mycursor = mydb.cursor()


mycursor.execute("DROP TABLE IF EXISTS music")
mycursor.execute("CREATE TABLE music (id VARCHAR(255), address VARCHAR(255))")

mycursor.execute("INSERT INTO music(id, address) VALUES ('000' ,'https://youtu.be/fiBvOKmuWKg'), ('001' ,'https://www.youtube.com/watch?v=Z5tEd16aBCY'), ('002' ,'https://youtu.be/GGU1P6lBW6Q'), \
                                                        ('010' ,'https://www.youtube.com/watch?v=DGQwd1_dpuc'), ('011' ,'https://www.youtube.com/watch?v=ysVBEAfkg1s'), ('012' ,'https://www.youtube.com/watch?v=4S4ahVVAGDo'), \
                                                        ('020' ,'https://youtu.be/P2u4G9IccCw'), ('021' ,'https://youtu.be/60toIb4oc_Y'), \
                                                        ('022' ,'https://www.youtube.com/watch?v=f71YVMsQHuI'), ('100' ,'https://www.youtube.com/watch?v=FtJXWw-ommA'), \
                                                        ('101' ,'https://youtu.be/sVx1mJDeUjY'), ('102' ,'https://youtu.be/aXJhM0rY8AQ'), \
                                                        ('110' ,'https://www.youtube.com/watch?v=4vgcYBwyw28'), ('111' ,'https://www.youtube.com/watch?v=V1bFr2SWP1I'), \
                                                        ('112' ,'https://youtu.be/zbL_I65VAcE'), ('120' ,'https://youtu.be/LSiooa1Kym0'), \
                                                        ('121' ,'https://youtu.be/LBt60dfwEBY'), ('122' ,'https://www.youtube.com/watch?v=68ugkg9RePc')")

#code creation
#FACE . COLOR . NOISE
def createId(img):
    id = detectFace(img)
    color = avgColor(img)
    
    if(color[0] > color[1] and color[0] > color[2]):
        id = id + '0' 
    if(color[1] > color[2] and color[1] > color[0]):
        id = id + '1'  
    if(color[2] > color[0] and color[2] > color[1]):
        id = id + '2'

    noise = estimate_noise(img)

    if(noise < 5):
        id = id + '0'
    if(noise >= 5 and noise < 5):
        id = id + '1'
    if(noise >= 10):
        id = id + '2'

    return id
    
alive = True

while(alive):
    msg = "Let's do something fun!"
    title = "Photo symphony"
    choices=['Upload Image','Take Photo','Exit']
    choice=buttonbox(msg, title,choices)

    if choice=='Upload Image':
        image = fileopenbox(msg="Choose an image", default=r"C:\Users\user\.atom\\")
        id = createId(image)

        mycursor.execute("SELECT address FROM music WHERE ID = " + id )
        link = mycursor.fetchall()[0][0]
        pywhatkit.playonyt(link)

    elif choice=='Take Photo':
        cam = cv2.VideoCapture(0)

        cv2.namedWindow("PRESS SPACE")

        while True:
            ret, frame = cam.read()
            if not ret:
                print("failed to grab frame")
                break
            cv2.imshow("PRESS SPACE", frame)

            k = cv2.waitKey(1)
            if k%256 == 27:
                # ESC pressed
                print("Escape hit, closing...")
                break
            elif k%256 == 32:
                # SPACE pressed
                img = "cheese.png".format()
                cv2.imwrite(img, frame)
                id = createId("cheese.png")

        cam.release()

        cv2.destroyAllWindows()

        mycursor.execute("SELECT address FROM music WHERE ID = " + id )
        link = mycursor.fetchall()[0][0]
        pywhatkit.playonyt(link)

    elif choice=='Exit':
        sys.exit
        alive = False



